In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-20"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
12746,2024-04-20,Brasil Nbb,17:10,Pinheiros,R10 Score Vasco da Gama,1.82,1.90,146.5,1.85,1.85,-1.5,1.94,2.08,CQfLrMQ0,0.549451,0.526316,0.540541,0.540541,0.075766,0.4,0.4,NaN,NaN,136.088,21.747155,0.159802,130.912,40.629242,0.310355,163.232,196.048,135.52,131.79,0.0,0.0,0.0,0.0,0.030413,0.000000,0.049251,-1.56,-0.312,-2.628205,0.582598,0.4,-0.182598,-0.85,-0.170,-5.294118,0.446539,0.5,0.053461
12747,2024-04-20,Brasil Nbb,19:30,Botafogo,Flamengo,6.41,1.09,166.5,1.84,1.86,12.5,2.03,1.01,jXsmcLuf,0.156006,0.917431,0.543478,0.537634,0.073437,0.2,0.8,NaN,NaN,282.312,83.387021,0.295372,101.102,10.714958,0.105982,235.636,107.384,202.74,90.30,0.0,0.0,0.0,0.0,1.003149,0.007644,0.474506,-2.82,-0.564,-9.592199,0.409097,0.4,-0.009097,-1.73,-0.346,-0.260116,0.832546,0.8,-0.032546
12748,2024-04-20,Espanha Acb,15:45,Palencia,Granada,1.72,2.14,165.5,1.81,1.95,-3.5,1.95,2.51,4bOeEF8H,0.581395,0.467290,0.552486,0.512821,0.048685,0.2,0.2,NaN,NaN,414.636,383.777530,0.925577,205.158,106.883518,0.520981,225.900,386.880,158.62,393.90,0.0,0.0,0.0,0.0,0.153878,0.052657,0.177569,2.10,0.420,1.714286,0.000000,0.0,0.000000,-0.96,-0.192,-5.937500,0.000000,0.0,0.000000
12749,2024-04-20,Eua Nba,16:30,Minnesota Timberwolves,Phoenix Suns,1.81,2.06,213.5,1.80,1.95,-2.5,2.00,2.35,IawbqvF7,0.552486,0.485437,0.555556,0.512821,0.037923,0.6,0.8,NaN,NaN,203.656,29.049817,0.142642,188.070,44.880452,0.238637,195.582,213.534,189.74,261.25,0.0,0.0,0.0,0.0,0.091357,0.056569,0.113787,-3.27,-0.654,-1.238532,0.665373,0.3,-0.365373,2.15,0.430,2.465116,0.528455,0.5,-0.028455
12750,2024-04-20,Eua Nba,19:00,New York Knicks,Philadelphia 76ers,1.66,2.31,208.5,1.80,1.95,-4.5,2.05,3.15,QNFGHB8a,0.602410,0.432900,0.555556,0.512821,0.035310,0.4,0.6,NaN,NaN,198.360,4.969024,0.025051,214.832,52.539171,0.244559,199.624,293.936,196.80,181.90,0.0,0.0,0.0,0.0,0.231546,0.056569,0.299161,-1.62,-0.324,-2.037037,0.599867,0.6,0.000133,3.38,0.676,1.937870,0.554062,0.5,-0.054062
12751,2024-04-20,Eua Nba,21:30,Denver Nuggets,Los Angeles Lakers,1.34,3.43,223.5,1.80,1.95,-8.5,2.00,7.50,2RsPjU8c,0.746269,0.291545,0.555556,0.512821,0.037814,0.4,0.2,NaN,NaN,189.438,29.043226,0.153313,199.984,22.103228,0.110525,160.534,215.450,233.45,212.30,0.0,0.0,0.0,0.0,0.619645,0.056569,0.818755,0.82,0.164,2.073171,0.762571,0.8,0.037429,0.92,0.184,13.206522,0.510022,0.6,0.089978
12752,2024-04-20,Alemanha Bbl,13:30,Hamburg,Ludwigsburg,2.31,1.62,170.5,1.86,1.94,2.5,1.96,1.48,0ELMiNcj,0.432900,0.617284,0.537634,0.515464,0.050184,0.2,0.4,NaN,NaN,180.886,73.651914,0.407173,131.758,34.065511,0.258546,135.346,151.868,97.96,188.34,0.0,0.0,0.0,0.0,0.248297,0.029773,0.197332,-2.15,-0.430,-3.046512,0.000000,0.0,0.000000,-1.53,-0.306,-2.026144,0.560082,0.6,0.039918
12753,2024-04-20,Alemanha Pro A,14:00,Kirchheim,Frankfurt,1.98,1.75,153.5,1.87,1.79,1.5,1.85,1.61,I3XOS7cA,0.505051,0.571429,0.534759,0.558659,0.076479,0.2,0.4,NaN,NaN,185.014,60.431417,0.326632,113.320,22.482914,0.198402,131.544,110.908,192.50,113.10,0.0,0.0,0.0,0.0,0.087204,0.030912,0.098096,-0.02,-0.004,-245.000000,0.000000,0.0,0.000000,0.46,0.092,8.152174,0.000000,0.0,0.000000
12754,2024-04-20,Austrália Nbl1 Norte,07:00,Southern District,Ipswich Force,2.02,1.72,181.5,1.83,1.87,1.5,1.87,1.59,6R6egiz1,0.495050,0.581395,0.546448,0.534759,0.076445,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.113440,0.015289,0.114445,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.00

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,17:10,Brasil Nbb,Pinheiros,R10 Score Vasco da Gama,146.5,1.85,1.0000,Over
1,15:45,Espanha Acb,Palencia,Granada,165.5,1.81,1.0000,Over
2,16:30,Eua Nba,Minnesota Timberwolves,Phoenix Suns,213.5,1.80,1.0000,Over
3,19:00,Eua Nba,New York Knicks,Philadelphia 76ers,208.5,1.80,1.0000,Over
4,21:30,Eua Nba,Denver Nuggets,Los Angeles Lakers,223.5,1.80,1.0000,Over
5,04:00,Austrália Nbl1 Sul,Geelong F,Frankston F,154.5,1.84,1.0000,Over
6,04:30,Austrália Nbl1 Sul,Nunawading F,Hobart Chargers F,149.5,1.85,1.0000,Over
7,06:00,Austrália Nbl1 Central,Central Districts Lions F,Sturt Sabres F,141.5,1.81,1.0000,Over
8,08:00,Austrália Nbl1 Oeste,East Perth Eagles,Perry Lakes Hawks,180.5,1.84,1.0000,Over
9,06:00,Austrália Nbl1 Oeste,East Perth Eagles F,Perry Lakes Hawks F,156.5,1.80,1.0000,Over
